In [1]:
import tensorflow as tf 
import matplotlib.pyplot as plt 

In [2]:
## 주피터 노트북에서 바로 텐서보드 출력하기

from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def
    
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:950px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [4]:
tf.reset_default_graph()

In [7]:
x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
W = tf.Variable(tf.random_normal([2, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
   # Initialize TensorFlow variables
   sess.run(tf.global_variables_initializer())

   for step in range(10001):
       sess.run(train, feed_dict={X: x_data, Y: y_data})
       if step % 100 == 0:
           print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run(W))

   # Accuracy report
   h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
   print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)


0 0.85150677 [[0.48912716]
 [0.21716292]]
100 0.71004725 [[0.5667292 ]
 [0.41688922]]
200 0.70090663 [[0.37925234]
 [0.29910326]]
300 0.69669616 [[0.25117007]
 [0.2083324 ]]
400 0.6947657 [[0.16674618]
 [0.14385693]]
500 0.6938845 [[0.11100728]
 [0.09877842]]
600 0.693483 [[0.07408699]
 [0.06755389]]
700 0.6933001 [[0.04955294]
 [0.04606277]]
800 0.69321686 [[0.03320288]
 [0.03133837]]
900 0.69317895 [[0.02228016]
 [0.02128411]]
1000 0.69316167 [[0.01496838]
 [0.01443629]]
1100 0.69315374 [[0.01006574]
 [0.00978149]]
1200 0.6931502 [[0.00677402]
 [0.00662217]]
1300 0.6931486 [[0.00456154]
 [0.00448041]]
1400 0.6931479 [[0.00307319]
 [0.00302983]]
1500 0.69314754 [[0.00207124]
 [0.00204809]]
1600 0.6931474 [[0.00139639]
 [0.00138401]]
1700 0.69314724 [[0.00094167]
 [0.00093505]]
1800 0.69314724 [[0.0006351 ]
 [0.00063156]]
1900 0.6931472 [[0.00042849]
 [0.0004266 ]]
2000 0.6931472 [[0.00028906]
 [0.00028805]]
2100 0.6931472 [[0.00019503]
 [0.00019449]]
2200 0.6931472 [[0.00013159]
 [0.0

In [8]:
show_graph(tf.get_default_graph().as_graph_def())

In [9]:
tf.reset_default_graph()

In [26]:
x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
b1 = tf.Variable(tf.random_normal([2]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
b2 = tf.Variable(tf.random_normal([1]), name='bias2')
hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
# Launch graph
with tf.Session() as sess:
   # Initialize TensorFlow variables
   sess.run(tf.global_variables_initializer())
   for step in range(10001):
       sess.run(train, feed_dict={X: x_data, Y: y_data})
       if step % 100 == 0:
           print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run([W1, W2]))

   # Accuracy report
   h, p, a = sess.run([hypothesis, predicted, accuracy],
                      feed_dict={X: x_data, Y: y_data})
   print("\nHypothesis: ", h, "\nOne Hot Encoding: ", p, "Real Data: ", y_data, "\nAccuracy: ", a)


0 0.78963923 [array([[ 0.6070908 ,  1.1163998 ],
       [-1.1434671 , -0.17159672]], dtype=float32), array([[0.46779284],
       [0.6212906 ]], dtype=float32)]
100 0.69332385 [array([[ 0.5799458,  1.0881102],
       [-1.1767036, -0.1567447]], dtype=float32), array([[0.24266982],
       [0.24144748]], dtype=float32)]
200 0.6931071 [array([[ 0.5775972 ,  1.0823901 ],
       [-1.1896455 , -0.13172863]], dtype=float32), array([[0.25179648],
       [0.21559614]], dtype=float32)]
300 0.69292057 [array([[ 0.5766987 ,  1.0779653 ],
       [-1.2033224 , -0.10858399]], dtype=float32), array([[0.26574743],
       [0.1967231 ]], dtype=float32)]
400 0.6927382 [array([[ 0.5771738 ,  1.0744526 ],
       [-1.2180501 , -0.08717312]], dtype=float32), array([[0.28137645],
       [0.17970264]], dtype=float32)]
500 0.6925535 [array([[ 0.57920027,  1.0716792 ],
       [-1.2339509 , -0.06734625]], dtype=float32), array([[0.29870138],
       [0.16427898]], dtype=float32)]
600 0.6923603 [array([[ 0.5829913 ,  

5200 0.08690167 [array([[ 5.257835 ,  4.528931 ],
       [-5.3793774, -4.187979 ]], dtype=float32), array([[ 6.8295956],
       [-6.035688 ]], dtype=float32)]
5300 0.08087324 [array([[ 5.303411 ,  4.60594  ],
       [-5.422592 , -4.2678504]], dtype=float32), array([[ 6.949028 ],
       [-6.1812053]], dtype=float32)]
5400 0.07555729 [array([[ 5.3460054,  4.677372 ],
       [-5.463345 , -4.341808 ]], dtype=float32), array([[ 7.062716],
       [-6.317955]], dtype=float32)]
5500 0.07084259 [array([[ 5.3859377,  4.7438583],
       [-5.5018606, -4.4105372]], dtype=float32), array([[ 7.1711063],
       [-6.4468255]], dtype=float32)]
5600 0.06663811 [array([[ 5.4234867,  4.805952 ],
       [-5.538341 , -4.474643 ]], dtype=float32), array([[ 7.2746153],
       [-6.5685954]], dtype=float32)]
5700 0.062869914 [array([[ 5.4588923,  4.864118 ],
       [-5.572967 , -4.534623 ]], dtype=float32), array([[ 7.3736033],
       [-6.6839395]], dtype=float32)]
5800 0.059476644 [array([[ 5.4923587,  4.918762

In [11]:
show_graph(tf.get_default_graph().as_graph_def())

# 텐서보드 이용해서 코스트 등 지표 저장 및 출력

In [3]:
from datetime import datetime
from datetime import timezone

def utc_to_local(utc_dt):
    return utc_dt.replace(tzinfo=timezone.utc).astimezone(tz=timedelta(hours=9))

now = datetime.now().strftime("%Y%m%d%H%M%S")
root_logdir = "logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [7]:
tf.reset_default_graph()

tf.set_random_seed(777)

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope("Layer1"):

    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    tf.summary.histogram("W1", W1)
    tf.summary.histogram("b1", b1)
    tf.summary.histogram("Layer1", layer1)
    
with tf.name_scope("Layer2"):
    
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    tf.summary.histogram("W2", W2)
    tf.summary.histogram("b2", b2)
    tf.summary.histogram("Hypothesis", hypothesis)
    
# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
# Accuracy computation

# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)

# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(logdir)
    writer.add_graph(sess.graph)  
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    
    
    # Show the graphfor step in range(10001):
    
    
    for step in range(10001):
        _, summary, cost_val = sess.run(
            [train, merged_summary, cost], feed_dict={X: x_data, Y: y_data}
        )
        writer.add_summary(summary, global_step=step)

        if step % 100 == 0:
            print(step, cost_val)


# Accuracy report
    h, p, a = sess.run([hypothesis, predicted, accuracy],
                      feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nOne Hot: ", p, "Real Data: ", y_data, "\nAccuracy: ", a)

    

0 0.7168677
100 0.7048352
200 0.7006425
300 0.69828176
400 0.6968283
500 0.69583476
600 0.6950847
700 0.69446766
800 0.69392145
900 0.6934054
1000 0.692889
1100 0.6923442
1200 0.6917429
1300 0.69105333
1400 0.690238
1500 0.6892523
1600 0.6880418
1700 0.6865416
1800 0.684674
1900 0.6823479
2000 0.6794574
2100 0.6758829
2200 0.67149496
2300 0.6661666
2400 0.65979385
2500 0.65232533
2600 0.6437934
2700 0.6343328
2800 0.6241759
2900 0.6136205
3000 0.60298
3100 0.59253466
3200 0.58249635
3300 0.572994
3400 0.5640751
3500 0.55571383
3600 0.5478195
3700 0.54023814
3800 0.53273976
3900 0.52498734
4000 0.5164845
4100 0.50651914
4200 0.49418992
4300 0.47861344
4400 0.45913985
4500 0.4352355
4600 0.4064455
4700 0.37292784
4800 0.33605057
4900 0.29825956
5000 0.26215848
5100 0.22959006
5200 0.20134875
5300 0.17742273
5400 0.15737101
5500 0.14060763
5600 0.12655546
5700 0.11470971
5800 0.10465306
5900 0.09604967
6000 0.088632524
6100 0.08218974
6200 0.07655333
6300 0.071589336
6400 0.067190185
6500

In [24]:
show_graph(tf.get_default_graph().as_graph_def())

In [32]:
tf.reset_default_graph()

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope("Layer1"):

    W1 = tf.Variable(tf.random_normal([2, 10]), name='weight1')
    b1 = tf.Variable(tf.random_normal([10]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    tf.summary.histogram("W1", W1)
    tf.summary.histogram("b1", b1)
    tf.summary.histogram("Layer1", layer1)
    
with tf.name_scope("Layer2"):
    
    W2 = tf.Variable(tf.random_normal([10, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    tf.summary.histogram("W2", W2)
    tf.summary.histogram("b2", b2)
    tf.summary.histogram("Hypothesis", hypothesis)
    
# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
# Accuracy computation

# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)

# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(logdir)
    writer.add_graph(sess.graph)  
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    
    
    # Show the graphfor step in range(10001):
    
    
    for step in range(10001):
        _, summary, cost_val = sess.run(
            [train, merged_summary, cost], feed_dict={X: x_data, Y: y_data}
        )
        writer.add_summary(summary, global_step=step)

        if step % 500 == 0:
            print(step, cost_val)



   # Accuracy report
    h, p, a = sess.run([hypothesis, predicted, accuracy],
                      feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis:\n", h, "\nOne Hot:\n", p, "\nReal Data:\n", y_data, "\nAccuracy:", a)

0 1.7557437
500 0.59752345
1000 0.4159096
1500 0.22878583
2000 0.12973022
2500 0.08258954
3000 0.058053493
3500 0.04377957
4000 0.034692533
4500 0.028497312
5000 0.024046987
5500 0.020717632
6000 0.018145077
6500 0.01610482
7000 0.014451478
7500 0.013087399
8000 0.011944611
8500 0.010974696
9000 0.010142129
9500 0.009420318
10000 0.008789082

Hypothesis:
  [[0.00890001]
 [0.99284583]
 [0.9894337 ]
 [0.008374  ]] 
One Hot:
  [[0.]
 [1.]
 [1.]
 [0.]] 
Real Data:
  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0
